In [1]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
total_df=pd.read_csv("risk_data/Train_data_set.csv", encoding='utf-8-sig')

In [3]:
total_df.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량,위험도
0,1,1,0,126690,0,-1.8,0.0,7.0
1,1,1,0,126690,10,-1.8,0.0,7.0
2,1,1,0,126690,20,-1.8,0.0,7.0
3,1,1,0,126690,30,-1.8,0.0,7.0
4,1,1,0,126690,40,-1.8,0.0,7.0


### pycaret을 이용하여 머신러닝 모델 선정
    1. Data가 커서 최적 모델 선정에 있어 많은 시간이 걸려 보류

In [5]:
from pycaret.classification import *
from time import time

In [35]:
total_df['위험도']=total_df['위험도'].astype('str')

In [36]:
cell_start_time = time()
clf = setup(data = total_df, target='위험도', silent=True)
cell_end_time = time()
print("CELL RUN TIME : ",cell_end_time - cell_start_time)

,Description,Value
0,session_id,4346
1,Target,위험도
2,Target Type,Multiclass
3,Label Encoded,"0.0: 0, 0.3: 1, 0.6: 2, 0.9: 3, 1.2: 4, 1.4: 5, 1.7: 6, 10.0: 7, 2.0: 8, 2.1: 9, 2.3: 10, 2.5: 11, 2.6: 12, 2.7: 13, 2.8: 14, 2.9: 15, 3.0: 16, 3.1: 17, 3.2: 18, 3.3: 19, 3.4: 20, 3.5: 21, 3.7: 22, 3.8: 23, 3.9: 24, 4.1: 25, 4.2: 26, 4.4: 27, 4.5: 28, 4.6: 29, 4.7: 30, 4.8: 31, 5.0: 32, 5.1: 33, 5.2: 34, 5.3: 35, 5.4: 36, 5.5: 37, 5.6: 38, 5.7: 39, 5.8: 40, 5.9: 41, 6.0: 42, 6.2: 43, 6.3: 44, 6.4: 45, 6.6: 46, 6.7: 47, 6.9: 48, 7.0: 49, 7.1: 50, 7.2: 51, 7.3: 52, 7.5: 53, 7.6: 54, 7.7: 55, 7.9: 56, 8.0: 57, 8.2: 58, 8.3: 59, 8.5: 60, 8.8: 61, 9.1: 62, 9.4: 63, 9.7: 64"
4,Original Data,"(2448189, 8)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


CELL RUN TIME :  22.559667348861694


In [ ]:
# 모델 성능 비교  # 잠정 보류
cell_start_time = time()
top5_models = compare_models(sort = 'Accuracy', n_select = 5)
cell_end_time = time()
print("CELL RUN TIME : ",cell_end_time - cell_start_time)

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:21:39
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,1.0000,1.0000,0.9983,1.0000,1.0000,1.0000,1.0000,73.5470
dt,Decision Tree Classifier,0.9999,0.9999,0.9975,0.9999,0.9999,0.9999,0.9999,3.0070
knn,K Neighbors Classifier,0.9150,0.9967,0.7714,0.9142,0.9144,0.9046,0.9046,2028.1280
ridge,Ridge Classifier,0.3916,0.0000,0.0375,0.2187,0.2505,0.2608,0.2919,1.3020
ada,Ada Boost Classifier,0.3550,0.6957,0.0298,0.1465,0.2033,0.2131,0.2608,94.2490
svm,SVM - Linear Kernel,0.2926,0.0000,0.0397,0.3162,0.2706,0.1962,0.2271,652.4590
lr,Logistic Regression,0.2437,0.6272,0.0166,0.1150,0.1152,0.0319,0.0558,2263.0430
nb,Naive Bayes,0.1879,0.8099,0.2047,0.3212,0.2237,0.1301,0.1356,6.2420
qda,Quadratic Discriminant Analysis,0.0019,0.0000,0.0154,0.0000,0.0000,0.0000,0.0000,4.1030


### RandomForest을 이용한 데이터 평가

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

In [14]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [15]:
train_data.head()

,month,day,시군구index,철새수,최초발생거리,평균온도,일강수량
0,1,1,0,126690,0,-1.8,0.0
1,1,1,0,126690,10,-1.8,0.0
2,1,1,0,126690,20,-1.8,0.0
3,1,1,0,126690,30,-1.8,0.0
4,1,1,0,126690,40,-1.8,0.0


In [16]:
target_data.head()

,위험도
0,7.0
1,7.0
2,7.0
3,7.0
4,7.0


In [17]:
# target_data one-hot-encoding
target_data2 = pd.get_dummies(target_data['위험도'])
target_data_result = pd.concat([target_data, target_data2], axis=1)

In [18]:
target_data2.head()

,0.0,0.3,0.6,0.9,1.2,1.4,1.7,10.0,2.0,2.1,2.3,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.7,3.8,3.9,4.1,4.2,4.4,4.5,4.6,4.7,4.8,5.0,5.1,5.2,5.3,5.4,5.5,5.6,5.7,5.8,5.9,6.0,6.2,6.3,6.4,6.6,6.7,6.9,7.0,7.1,7.2,7.3,7.5,7.6,7.7,7.9,8.0,8.2,8.3,8.5,8.8,9.1,9.4,9.7
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
train_x, test_x, train_y, test_y = train_test_split(train_data, target_data2, test_size = 0.2, random_state = 42)

In [20]:
clf = RandomForestClassifier(n_estimators=10, max_depth=10, random_state=42)
clf.fit(train_x,train_y)

RandomForestClassifier(max_depth=10, n_estimators=10, random_state=42)

In [21]:
predict1 = clf.predict(test_x)
print(accuracy_score(test_y,predict1))

0.7548168238576254


In [23]:
# 모델 저장
import joblib
joblib.dump(clf, 'model/Randomforest.pkl')

['model/Randomforest.pkl']

In [24]:
# 모델 불러오기
rf_model=joblib.load('model/Randomforest.pkl')

In [25]:
predict1 = rf_model.predict(test_x)
print(accuracy_score(test_y,predict1))

0.7548168238576254


### XGBoost Classifier 모델 이용

In [26]:
import xgboost

In [27]:
train_data=total_df.drop(['위험도'], axis=1)
target_data=total_df[['위험도']].astype('str')

In [28]:
train_X, test_X, train_Y, test_Y = train_test_split(train_data, target_data, test_size = 0.2, random_state = 42)

In [29]:
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(1958551, 7) (489638, 7) (1958551, 1) (489638, 1)


In [30]:
model = xgboost.XGBClassifier()
model.fit(train_X.values, train_Y.values)

[17:43:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [31]:
y_pred = model.predict(test_X.values)
#predictions = [np.round(value,1) for value in y_pred]

In [32]:
print("(pred == y_test) score: {0:.3f}".format((y_pred==test_Y['위험도'].values).mean()))

(pred == y_test) score: 0.909


In [33]:
# 모델 저장
import joblib
joblib.dump(model, 'model/xgboost.pkl')

['model/xgboost.pkl']

In [34]:
# 모델 불러오기
xgb_model=joblib.load('model/xgboost.pkl')